In [1]:
from google.colab import files
uploaded = files.upload()


import pandas as pd
import numpy as np
import io
data = pd.read_csv(io.BytesIO(uploaded['gyapu.csv']))


Saving gyapu.csv to gyapu.csv


In [2]:
import pandas as pd
import numpy as np
import io
data = pd.read_csv(io.BytesIO(uploaded['gyapu.csv']))


In [3]:
categories = data["extras.category"].value_counts(dropna=False)

In [4]:
categories = categories.to_frame()


In [5]:
categories.columns= ["Count"]


In [6]:
categories = categories.query("Count > 3")


In [7]:
data_truncated = data.drop(columns=["datetime_extracted", "extras.breadcrumb", "extras.sellerName", "image", "price", "url", "_id", "ecommerce"])
data_truncated

,description,extras.category,name
0,"[""Boosts Immunity & Metabolism"",""Anti-Ageing P...",Dry & Packaged Food,Desi Grub Raw White Honey | Unpasteurized Unfi...
1,"[""Improves Hearth Health"",""Moisturizes dry/tri...",Dry & Packaged Food,Desi Grub Apricot Kernel Oil Extra Virgin - 10...
2,"[""TOP CLASS QUALITY - Our towels are made of r...",Hygiene & Cleaning Essentials,12 Pcs Kitchen Dish Towels Super Absorbent Ki...
3,"[""TOP CLASS QUALITY - Our towels are made of r...",Hygiene & Cleaning Essentials,6 Pcs Kitchen Dish Towels Super Absorbent Kit...
4,"[""【LINT-FREE, SCRATCH-FREE, SWIRL-FREE, 】Car W...",Hygiene & Cleaning Essentials,Multicolor Waterproof Free Size Microfiber Han...
...,...,...,...
12052,[],NaN,Air Wick Nagpur Narangi L&O Refill 250ml
12053,"[""Daily Protection: Dettol Bathing soap bar pr...",NaN,Dettol Soap Original 125gm
12054,[],NaN,Tokla Tea 50 Bags 100Gm
12055,"[""India's No.1 Floor Cleaning Brand. Recommend...",NaN,LIzol Floral 500ml


In [8]:
category_names = list(categories.index)
data_truncated = data_truncated.rename(columns = {"extras.category": "category"})
filtered_data = data_truncated
data_truncated = data_truncated.dropna()
queryset = [f'category.str.startswith("{name}")' for name in category_names]
filtered_data = data_truncated.query(" | ".join(queryset), engine="python")

filtered_data["description"] = filtered_data["description"].str.replace("[", "")
filtered_data["description"] = filtered_data["description"].str.replace("]", "")
filtered_data["description"] = filtered_data["description"].str.replace('"', "")



<ipython-input-8-53f1e60aa6b3>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  filtered_data["description"] = filtered_data["description"].str.replace("[", "")
<ipython-input-8-53f1e60aa6b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["description"] = filtered_data["description"].str.replace("[", "")
<ipython-input-8-53f1e60aa6b3>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  filtered_data["description"] = filtere

In [9]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42, shuffle=True)


In [10]:
train_data

,description,category,name
2252,Akshar Parichaya is a nepali books that introd...,Children's,Combo Of Pre-School Nepali Kids Books Set
299,"Colin Eco Clean 5ltr ,Ultra shine formula for ...",Hygiene Essentials,Colin Eco Clean 5ltr 1jar
5819,"Ok Play Square Table Wooden,Product Dimensions...",Baby Furniture,Ok Play Square Table Wooden
6157,"Battery capacity: 1200mAh.,A soft-lighting lam...",Table Lamps,Remax KwangChe Series LED Lamp RT-L01
6563,Asana Pranayama Mudra Bandha By Swami Satyanan...,Books,Asana Pranayama Mudra Bandha By Swami Satyanan...
...,...,...,...
5291,"Language: Nepali,Binding: Paperback,Publisher:...",Business,How To Attract Money (Nepali)
5326,"Hardy’s Nottage Hill Chardonnay White Wine ,Al...",Australian Wines,Hardys Nottage Hill Chardonnay White Wine 750ml
5493,"Little Genius Tool Box,Product Dimensions : 27...",Toys & Games,Little Genius Tool Box
869,"Nutritious and healthy,Store in cool dry place...",Groceries,Lindt Swiss Milk Raisen Nut 100gm


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])


In [12]:
from sklearn import preprocessing
encoder = preprocessing.OrdinalEncoder()

In [13]:
encoder.fit(np.reshape(filtered_data.category.to_numpy(), (len(filtered_data.category), 1)))
Y = encoder.transform(np.reshape(train_data.category.to_numpy(), (len(train_data.category), 1)))

In [14]:
Y = Y.reshape(-1)

In [15]:
text_clf.fit(train_data.description.to_numpy(), Y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [16]:
encoder.inverse_transform([text_clf.predict(["Premium wine"])])

array([['Wine']], dtype=object)

In [17]:
predicted = text_clf.predict(test_data.description)

In [18]:
predicted

array([180.,  49., 180., ..., 180.,  31.,  45.])

In [19]:
target = encoder.transform(np.reshape(test_data.category.to_numpy(), (len(test_data.category), 1)))

In [20]:
from sklearn import metrics
print(metrics.classification_report(target, predicted,))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         4
         1.0       1.00      0.60      0.75        15
         3.0       0.00      0.00      0.00         6
         5.0       0.00      0.00      0.00         6
         6.0       0.00      0.00      0.00         9
         7.0       0.00      0.00      0.00        13
         8.0       0.00      0.00      0.00         5
         9.0       0.00      0.00      0.00         5
        10.0       0.00      0.00      0.00         7
        11.0       0.00      0.00      0.00         3
        12.0       0.00      0.00      0.00         7
        13.0       0.00      0.00      0.00         3
        15.0       0.00      0.00      0.00         2
        19.0       0.00      0.00      0.00         2
        21.0       0.00      0.00      0.00         3
        22.0       1.00      0.33      0.50         9
        23.0       0.00      0.00      0.00         6
        24.0       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
list(encoder.categories_[0])

[' Bags & Luggage ',
 ' Footwear',
 'Action Figures & Collectibles',
 'Air Treatment',
 'Anniversary',
 'Arts & Craft for Kids',
 'Audio',
 'Australian Wines',
 'Auto Accessories',
 'Baby Clothing',
 'Baby Furniture',
 'Baby Gear',
 'Baby Health & Hygiene',
 'Baby Hygiene Material',
 'Baby Oil',
 'Baby Safety & Accessories',
 'Baby Shampoo',
 'Baby Shower',
 'Baby Strollers',
 'Baby Swing',
 'Backpacks & Carriers',
 'Bags & Shoes',
 'Bags & luggage',
 'Bathing & Gromming ',
 'Bathroom Safety',
 'Beauty & Health ',
 'Bedding and Its Accessories',
 'Beer',
 'Beverages ',
 'Biography',
 'Birthday',
 'Books',
 'Bottle',
 'Bottle Feeding',
 'Brandy',
 'Bulbs',
 'Business',
 'Cables',
 'Champagne',
 "Children's",
 'Chilean Wines',
 'Clock',
 'Clothing ',
 'Clothing Sets',
 'Consumer Electronics',
 'Cooking Essentials ',
 'Cooktops & Ranges',
 'Coolpad ',
 'Covid ',
 'Covid-19 Safety Gears',
 'Dairy Products ',
 'Decoration Lights',
 'Diaper Bags',
 'Diapers & Potty',
 'Disposable Diapers',
 

In [22]:
from sklearn.model_selection import train_test_split

filtered_data["combined"] = filtered_data["name"] + " " + filtered_data["description"] 

train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42, shuffle=True)


<ipython-input-22-8eecedc976e8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["combined"] = filtered_data["name"] + " " + filtered_data["description"]


In [23]:
train_data

,description,category,name,combined
2252,Akshar Parichaya is a nepali books that introd...,Children's,Combo Of Pre-School Nepali Kids Books Set,Combo Of Pre-School Nepali Kids Books Set Aksh...
299,"Colin Eco Clean 5ltr ,Ultra shine formula for ...",Hygiene Essentials,Colin Eco Clean 5ltr 1jar,Colin Eco Clean 5ltr 1jar Colin Eco Clean 5ltr...
5819,"Ok Play Square Table Wooden,Product Dimensions...",Baby Furniture,Ok Play Square Table Wooden,Ok Play Square Table Wooden Ok Play Square Tab...
6157,"Battery capacity: 1200mAh.,A soft-lighting lam...",Table Lamps,Remax KwangChe Series LED Lamp RT-L01,Remax KwangChe Series LED Lamp RT-L01 Battery ...
6563,Asana Pranayama Mudra Bandha By Swami Satyanan...,Books,Asana Pranayama Mudra Bandha By Swami Satyanan...,Asana Pranayama Mudra Bandha By Swami Satyanan...
...,...,...,...,...
5291,"Language: Nepali,Binding: Paperback,Publisher:...",Business,How To Attract Money (Nepali),How To Attract Money (Nepali) Language: Nepali...
5326,"Hardy’s Nottage Hill Chardonnay White Wine ,Al...",Australian Wines,Hardys Nottage Hill Chardonnay White Wine 750ml,Hardys Nottage Hill Chardonnay White Wine 750m...
5493,"Little Genius Tool Box,Product Dimensions : 27...",Toys & Games,Little Genius Tool Box,"Little Genius Tool Box Little Genius Tool Box,..."
869,"Nutritious and healthy,Store in cool dry place...",Groceries,Lindt Swiss Milk Raisen Nut 100gm,Lindt Swiss Milk Raisen Nut 100gm Nutritious a...


In [24]:
encoder.fit(np.reshape(filtered_data.category.to_numpy(), (len(filtered_data.category), 1)))
Y = encoder.transform(np.reshape(train_data.category.to_numpy(), (len(train_data.category), 1)))

In [25]:
Y = Y.reshape(-1)

In [26]:
text_clf.fit(train_data.combined.to_numpy(), Y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [27]:
predicted = text_clf.predict(test_data.combined)

In [28]:
target = encoder.transform(np.reshape(test_data.category.to_numpy(), (len(test_data.category), 1)))

In [29]:
from sklearn import metrics
print(metrics.classification_report(target, predicted, zero_division = True))


              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00         4
         1.0       1.00      0.80      0.89        15
         3.0       1.00      0.17      0.29         6
         5.0       1.00      0.00      0.00         6
         6.0       1.00      0.00      0.00         9
         7.0       0.86      0.46      0.60        13
         8.0       1.00      0.00      0.00         5
         9.0       1.00      0.00      0.00         5
        10.0       1.00      0.00      0.00         7
        11.0       1.00      0.00      0.00         3
        12.0       1.00      0.00      0.00         7
        13.0       1.00      0.00      0.00         3
        15.0       1.00      0.00      0.00         2
        19.0       1.00      0.00      0.00         2
        21.0       1.00      0.00      0.00         3
        22.0       1.00      0.44      0.62         9
        23.0       1.00      0.00      0.00         6
        24.0       1.00    

In [30]:
predicted_names = text_clf.predict(test_data.name)
from sklearn import metrics
print(metrics.classification_report(target, predicted_names, zero_division=True))

              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00         4
         1.0       1.00      0.73      0.85        15
         3.0       1.00      0.50      0.67         6
         5.0       1.00      0.00      0.00         6
         6.0       1.00      0.00      0.00         9
         7.0       0.90      0.69      0.78        13
         8.0       1.00      0.00      0.00         5
         9.0       1.00      0.00      0.00         5
        10.0       1.00      0.00      0.00         7
        11.0       1.00      0.00      0.00         3
        12.0       1.00      0.00      0.00         7
        13.0       1.00      0.00      0.00         3
        15.0       1.00      0.00      0.00         2
        19.0       1.00      0.00      0.00         2
        21.0       1.00      0.00      0.00         3
        22.0       0.67      0.22      0.33         9
        23.0       1.00      0.00      0.00         6
        24.0       1.00    

In [31]:
predicted_names = text_clf.predict(test_data.name)
from sklearn import metrics
print(metrics.classification_report(target, predicted_names, zero_division = True))

              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00         4
         1.0       1.00      0.73      0.85        15
         3.0       1.00      0.50      0.67         6
         5.0       1.00      0.00      0.00         6
         6.0       1.00      0.00      0.00         9
         7.0       0.90      0.69      0.78        13
         8.0       1.00      0.00      0.00         5
         9.0       1.00      0.00      0.00         5
        10.0       1.00      0.00      0.00         7
        11.0       1.00      0.00      0.00         3
        12.0       1.00      0.00      0.00         7
        13.0       1.00      0.00      0.00         3
        15.0       1.00      0.00      0.00         2
        19.0       1.00      0.00      0.00         2
        21.0       1.00      0.00      0.00         3
        22.0       0.67      0.22      0.33         9
        23.0       1.00      0.00      0.00         6
        24.0       1.00    

In [ ]:
from joblib import dump

In [ ]:
dump(text_clf, 'classification_model1.joblib') 


['classification_model1.joblib']

In [ ]:
files.download('classification_model1.joblib')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dump(encoder, 'category_encoder1.joblib')

['category_encoder1.joblib']

In [ ]:
files.download('category_encoder1.joblib')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

In [35]:
text_clf.fit(train_data.combined.to_numpy(), Y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [36]:
text_clf.predict(["wine"])

array([194.])

In [38]:
predicted_names = text_clf.predict(test_data.combined)
from sklearn import metrics
print(metrics.classification_report(target, predicted_names, zero_division = True))

              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00         4
         1.0       0.94      1.00      0.97        15
         3.0       0.71      0.83      0.77         6
         5.0       1.00      0.00      0.00         6
         6.0       1.00      0.00      0.00         9
         7.0       0.80      0.92      0.86        13
         8.0       0.33      0.20      0.25         5
         9.0       0.67      0.40      0.50         5
        10.0       1.00      0.00      0.00         7
        11.0       1.00      1.00      1.00         3
        12.0       0.50      0.14      0.22         7
        13.0       1.00      1.00      1.00         3
        15.0       1.00      0.00      0.00         2
        19.0       1.00      0.00      0.00         2
        21.0       0.67      0.67      0.67         3
        22.0       0.75      1.00      0.86         9
        23.0       1.00      1.00      1.00         6
        24.0       1.00    

In [39]:
predicted_names = text_clf.predict(test_data.name)
from sklearn import metrics
print(metrics.classification_report(target, predicted_names, zero_division = True))

              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00         4
         1.0       0.94      1.00      0.97        15
         3.0       0.83      0.83      0.83         6
         5.0       1.00      0.17      0.29         6
         6.0       1.00      0.00      0.00         9
         7.0       0.80      0.92      0.86        13
         8.0       0.50      0.40      0.44         5
         9.0       0.50      0.20      0.29         5
        10.0       0.00      0.00      0.00         7
        11.0       1.00      1.00      1.00         3
        12.0       1.00      0.29      0.44         7
        13.0       0.50      1.00      0.67         3
        15.0       1.00      0.00      0.00         2
        19.0       1.00      0.00      0.00         2
        21.0       0.67      0.67      0.67         3
        22.0       0.50      0.78      0.61         9
        23.0       1.00      0.67      0.80         6
        24.0       1.00    

In [40]:
predicted_names = text_clf.predict(test_data.description)
from sklearn import metrics
print(metrics.classification_report(target, predicted_names, zero_division = True))

              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00         4
         1.0       0.70      0.93      0.80        15
         3.0       0.71      0.83      0.77         6
         5.0       1.00      0.00      0.00         6
         6.0       1.00      0.00      0.00         9
         7.0       0.64      0.69      0.67        13
         8.0       0.25      0.20      0.22         5
         9.0       1.00      0.20      0.33         5
        10.0       1.00      0.00      0.00         7
        11.0       1.00      1.00      1.00         3
        12.0       0.33      0.14      0.20         7
        13.0       1.00      0.00      0.00         3
        15.0       1.00      0.00      0.00         2
        19.0       1.00      0.00      0.00         2
        21.0       0.50      0.33      0.40         3
        22.0       0.64      1.00      0.78         9
        23.0       1.00      0.67      0.80         6
        24.0       1.00    